**About** : This notebook is used to validate models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import timm
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *
from collections import Counter
from numerize.numerize import numerize

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from util.plots import *
from util.metrics import *

from data.dataset import Abdominal2DDataset
from data.transforms import get_transfos
from data.preparation import prepare_data
from util.plots import plot_sample
from util.logger import upload_to_kaggle

from inference.extract_features import kfold_inference, Config

### Data

In [ ]:
df_patient, df_img = prepare_data(DATA_PATH)

In [ ]:
if "fold" not in df_patient.columns:
    folds = pd.read_csv(DATA_PATH + "folds_4.csv")
    df_img = df_img.merge(folds)
    df_patient = df_patient.merge(folds)

### Extract features

- Todo : multi-gpu

In [ ]:
USE_FP16 = True
SAVE = True

EXP_FOLDER = "../logs/2023-08-27/3/"

In [ ]:
# preds = kfold_inference(df_patient, df_img, EXP_FOLDER, use_fp16=USE_FP16, save=SAVE)

In [ ]:
FOLDS = [0, 1, 2, 3]

In [ ]:
preds = [np.load(EXP_FOLDER + f"pred_val_{fold}.npy") for fold in FOLDS]

### Eval

In [ ]:
df_val = df_img.copy()

In [ ]:
df_val['pred'] = -1

for i, tgt in enumerate(IMAGE_TARGETS):
    df_val[f"pred_{tgt}"] = -1
        
    for fold in FOLDS:
        df_val.loc[df_val[df_val['fold'] == fold].index, f"pred_{tgt}"] = preds[fold][:, i]

df_val = df_val[df_val['fold'] == 0]

In [ ]:
df_val_patient = df_val[['patient_id', 'pred_bowel_injury', 'pred_extravasation_injury']].groupby('patient_id').mean()
df_val_patient = df_val_patient.merge(df_patient, on="patient_id", how="left")

In [ ]:
for tgt in IMAGE_TARGETS:
    auc = roc_auc_score(df_val_patient[tgt], df_val_patient[f"pred_{tgt}"])
    print(f'- {tgt} auc : {auc:.3f}')

In [ ]:
plt.figure(figsize=(12, 4))
for i, tgt in enumerate(IMAGE_TARGETS):
    plt.subplot(1, 2, i + 1)
    sns.histplot(df_val_patient[f"pred_{tgt}"])
plt.show()

In [ ]:
losses, avg_loss = rsna_loss(df_val_patient[["pred_bowel_injury", "pred_extravasation_injury"]].values, df_val_patient)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")

In [ ]:
dummy = np.array([[0.04] * len(df_val_patient), [0.3] * len(df_val_patient)]).T
losses, avg_loss = rsna_loss(dummy, df_val_patient)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")

In [ ]:
dummy = np.array(
    [
        [0.04] * len(df_val_patient), [0.3] * len(df_val_patient),
        [0.6] * len(df_val_patient), [0.05] * len(df_val_patient), [0.05] * len(df_val_patient),
        [0.4] * len(df_val_patient), [0.07] * len(df_val_patient), [0.03] * len(df_val_patient),
        [0.3] * len(df_val_patient), [0.04] * len(df_val_patient), [0.07] * len(df_val_patient),
    ]
).T
losses, avg_loss = rsna_loss(dummy, df_val_patient)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
print(f'\n -> CV Score : {avg_loss :.3f}')

Done ! 